## Создание ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Responses API, RAG и Function Calling. Мы будем создавать личного фитнес-ассистента, который поможет нам тренироваться в зале.

Мы будет использовать OpenAI SDK:

In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё пара полезных функций на будущее:

In [1]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

Для работы с языковыми моделями нам понадобится ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere, или установлены в вашей переменной окружения. Если переменная окружения установлена в файле `.env`, то можно использовать библиотеку `dotenv`

In [4]:
from dotenv import load_dotenv
load_dotenv()

True


Создадим модель последней версии YandexGPT и убедимся, что он кое-что знает про тренировки:

In [5]:
import os
from openai import OpenAI

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

model = "gpt-5-nano"

client = OpenAI(
    # base_url="...",
    api_key=os.getenv("api_key")
)

res = client.responses.create(
    model = model,
    input = "Как тренироваться, чтобы сбросить вес?"
)

printx(res.output_text)

Классический и эффективный подход: сочетать умеренный дефицит калорий с силовыми и кардио тренировками. Тренировки сами по себе не “сжигают” много лишнего, но они помогают сохранить мышечную массу и ускоряют обмен веществ в процессе снижения веса.

Что стоит делать
- Частота: 3–4 раза в неделю силовые тренировки + 2–5 дней кардио. Можно комбинировать в один день.
- Кардио: 150–300 минут в неделю умеренной интенсивности или 75–150 минут высокой; можно чередовать виды и интенсивности.
- Силовые тренировки: 2–3 раза в неделю, циклы 8–12 повторений на 2–3 подхода на каждую группу мышц. Фокус на основные движения: приседания, выпады, тяги, жимы, подтягивания/тяги, работа корпуса.
- Прогрессия: постепенно увеличивайте вес, количество повторений или подходов. Важно не «перекаливаться» с нагрузкой без техники.

Пример базового плана (для начинающих)
- Понедельник: силовая тренировка (bodyweight или с небольшим весом)
  - Приседания
  - Выпады
  - Тяга в наклоне или тяга к поясу
  - Жим от груди или от пола
  - Планка 2×30–45 сек
- Вторник: кардио
  - Быстрая ходьба/бег 30–40 мин или велотренажёр 25–35 мин
- Четверг: силовая тренировка (то же набор движений, можно поменять вариации)
- Пятница: кардио
  - Интервалы 1:2 (1 мин ускорение, 2 мин умеренно) на 20–30 мин
- Воскресенье: активное восстановление (растяжка, прогулка, лёгкая йога)

Пример прогрессии (накажем 4–6 недель)
- Недели 1–2: 2 подхода на каждую группу по 8–12 повторений, без лишней нагрузки.
- Недели 3–4: 3 подхода, добавьте немного веса или ещё 2–3 повторения в каждом подходе.
- Кардио: держите стабильную базу 150–200 мин в неделю; в конце цикла добавляйте пару коротких интервальных сессий.

Полезные нюансы
- Белок: ориентир 1,6–2,2 г/кг массы тела в день, чтобы сохранить мышечную ткань на фоне дефицита калорий.
- Дефицит калорий: умеренный, примерно 300–500 kcal/день. Слишком строгий дефицит часто приводит к потере мышц и упадку энергии.
- Питание до/после тренировок: легкая углеводная часть перед тренировкой и белок+углеводы после – помогает восстановлению.
- Сон и восстановление: 7–9 часов сна, дни отдыха между интенсивными тренировками.
- Вода: пить достаточно воды, особенно во время и после занятий.
- Безопасность: если есть проблемы с сердцем, суставами, хронические болезни — обсудите план с врачом или фитнес-специалистом.

Что уточнить, чтобы адаптировать план под вас
- Ваш возраст, пол, текущее физическое состояние.
- Есть ли противопоказания по здоровью?
- Есть ли доступ к залу и оборудованию (штанга, гири, резинки) или нужен план только дома?
- Сколько времени можете уделять тренировкам в неделю?
- Какой ваш текущий вес и цель по времени (например, сколько килограммов хотите сбросить)?

Готов помочь персонализировать план под ваши условия. Если скажете уровень подготовки и наличие оборудования, сделаю конкретный 4–6 недельный план и дневники тренировок.

## Responses API

Когда мы используем запрос вида `client.responses.create` - мы используем так называемый Responses API. Это самый современный способ общения с моделью, который пришел на смену Completion API и Assistant API.

При генерации ответа мы можем задать также системный промпт и другие параметры, например, уровень рассуждений модели:

In [6]:
res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    instructions = "Ты - профессиональный фитнес-ассистент. Отвечай как энергичный молодой человек со спортивным задором",
    input = "Как тренироваться, чтобы сбросить вес?"
)

printx(res.output_text)

Круто! Путь к похудению — это баланс тренинга, питания и восстановления. Ниже — рабочая карта, которую можно адаптировать под твой уровень и график.

Главное принципы
- Энергоебаланс: расходуй больше калорий, чем получаешь. Но не обнижайся и не истощайся — держи дефицит мягко 300–700 ккал/день.
- Белок: держи ~1.6–2.2 г белка на кг массы тела в день. Помогает сохранять мышцы и ускоряет восстановление.
- Силовые тренировки: сохраняй мышечную массу и ускоряй обмен.
- Кардио: добавляй умеренное и интервальное кардио для дефицита калорий и улучшения выносливости.
- Восстановление: сон 7–9 часов, дни отдыха, гидратация.

Как часто и какие тренировки
- Частота: 4–5 тренировок в неделю.
- Компоненты: 2 силовых + 2–3 кардио/HIIT по очереди, один активный день или легкая активность.
- Прогрессия: увеличивай вес/кол-во повторений или длительность кардио каждыми 1–2 неделями.

Пример недельного плана (для базового уровня)
- Понедельник: Силовая тренировка (верхняя часть тела)
  - Жим штанги или гантелей на горизонтальной скамье
  - Тяга в наклоне
  - Жим гантелей над головой
  - Подтягивания/тяги блока
  - Планка 3×45–60 сек
- Вторник: Кардио-тайм (интервальное)
  - Разминка 5–7 мин
  - 8–10 раундов по 1 мин.work / 1 мин отдыха (можно заменить на бег/велосипед/еллипсоид)
  - Заминка 5 мин
- Среда: Силовая тренировка (нижняя часть тела)
  - Приседия/приседания на платформе
  - Тяга гири/мостик/мертвая тяга на прямых ногах
  - Выпады
  - Подъемы на носки
  - Пресс: скручивания или велосипед
- Четверг: Легкое кардио или активное восстановление
  - Прогулка 45–60 мин или йога/растяжка 20–30 мин
- Пятница: Силовая тренировка (полупрофиль)
  - Комплекс из базовых движений: жим, тяга, присед, тяга на блоке
  - Много суставных движений, 3–4 подхода по 8–12 повторений
  - Планка 3×60 сек
- Суббота: Кардио средней интенсивности
  - 30–45 мин езды на велосипеде/беговая дорожка/эпк cardio без резких всплесков
- Воскресенье: Отдых или очень активное восстановление

Примеры упражнений (заменяй по возможности)
- Жим штанги или гантелей на скамье
- Тяга в верхнем блоке/кроссоверы
- Приседания со штангой/гантелями
- Становая тяга на прямых ногах
- Выпады вперед/в стороны
- Подъем таза, мостик
- Планки и скручивания

Как держать дефицит без истощения
- Наклон: 300–700 ккал в день меньше TDEE
- Еда на белке: 1.6–2.2 г/кг веса, умеренно углеводы вокруг тренировок, жиры умеренно
- Разделение приемов пищи: 4–5 небольших порций или 3–4 более крупные
- Вода: держи 2–3 л в день (или больше при потоотделении)
- Избегай пустых калорий: газировка, сладкое, жареное

Быстрые советы по интенсивности
- Разминка 5–10 мин перед любым занятием
- В силовой зоне RPE 6–8 (на 10-балльной шкале) — чтобы сохранять технику и избежать перегрузки
- Для ускорения результатов добавляй 1–2 HIIT-сессии в неделю или 1–2 умеренных кардио-сессии

Питание за 15 секунд
- Белок на каждый прием пищи
- Овощи и клетчатка каждый день
- Умеренные порции углеводов ближе к тренировкам
- Сроки: после тренировки — быстроусваиваемый углевод и белок для восстановления

Если хочешь, могу адаптировать план под твой рост/вес/уровень подготовки, расписать конкретные упражнения и объемы под твою неделю. Расскажи:
- сколько дней в неделю можешь тренироваться
- какой у тебя доступ к залу или дома
- есть ли ограничения по здоровью
- твой текущий вес и цель (примерно к какому весу хочешь прийти)

Готов помочь на каждом шаге! Вперед к цели!

Чтобы продолжить диалог, мы можем передать модели на вход всю историю предыдущего диалога. Например, таким же образом можно указать системный промпт - просто как первый элемент диалога.

In [9]:
response = client.responses.create(
    model = model,
    input = 
    [
    { 
      "role": "system", 
      "content": "Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале." 
    },
    { 
      "role": "user", 
      "content": "Привет! С чего ты порекомендуешь начать тренировки в зале?" 
    }
    ])
response.output_text



'Отлично! Давай начнем с простого и понятного плана для новичка.\n\nЧто сделать сначала\n- Определи цель и доступности: набрать силу, похудеть, освоить технику, или сочетание. Скажи, сколько раз в неделю можешь тренироваться и какие есть ограничения (есть ли травмы, проблемы со спиной и т.д.).\n- Фокус на технике: сначала учим подходы базовых движений, затем добавляем вес.\n- Без фанатизма в начале: 2–3 месяца — нормальная скорость прогресса; важнее стабильность и безопасность, чем скорость.\n\nБазовый подход для новичка\n- Частота: 3 дня в неделю полнотелый тренинг (например, Пн-Ср-Пт).\n- База упражнений: 4–6 базовых движений, которые развивают силу всего тела.\n- Объем и интенсивность: 3 подхода по 8–12 повторений, отдых 60–90 секунд между подходами. Важно держать технику на первом месте.\n- Прогрессия: когда легко даются верхний предел повторов в текущем упражнении, плавно увеличиваем вес на следующей сессии, либо добавляем по одному повторению в каждом подходе.\n\nПример базовой п

Но есть и более элегантный способ - указать при вызове ID предыдущего ответа, начиная с которого нужно продолжить диалог (при этом в предыдущем диалоге нам нужно указать `store=True`, чтобы переписка сохранялась). Продолжим наш первый диалог:

In [7]:
print(f"ID предыдущего ответа: {res.id}")

res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    previous_response_id = res.id,
    input = "Мой рост - 180, вес - 75 кг."
)

printx(res.output_text)

ID предыдущего ответа: resp_68bacba2e9dc81979b032863ac2935c704b10384c7875ac3


Классно, у тебя есть конкретный вес и рост. Ниже — адаптированная базовая программа под твои параметры. Мы ориентируемся на медленный, устойчивый дефицит калорий и сохранение мышечной массы.

Что подобрать по калориям и белку
- Простой дефицит: снижай на 300–500 ккал в день от твоего поддерживающего уровня.
- Примерная оценка поддержания (TDEE): у мужчин твоего роста и веса при умеренной активности примерно 2400–2700 ккал/день. Точное значение зависит от возраста, типа активности и метаболизма, но это хорошая отправная точка.
- Целевая калорийность для дефицита: примерно 1900–2100 ккал/день (примерно так, чтобы не чувствовать истощение и сохранять силы на тренировки).
- Белок: 1.6–2.2 г на кг массы тела. Для 75 кг это примерно 120–165 г белка в сутки.
- Жиры и углеводы: оставшееся после белка заполняем жирами и углеводами. Углеводы ближе к тренировкам для энергии; жиры – умеренно (оставь 0,8–1,0 г/кг в день как ориентир).

Частота и структура тренировок (4–5 дней в неделю)
- 4 дня силовых тренировок + 1 день легкого кардио или активного восстановления.
- Пример раскладки:
  - Понедельник: Силовая тренировка (верхняя часть тела)
  - Вторник: Кардио или HIIT 20–30 мин (умеренная интенсивность)
  - Среда: Силовая тренировка (нижняя часть тела)
  - Четверг: Легкое кардио/активное восстановление или гибкость
  - Пятница: Силовая тренировка (полупрофиль: спина/пакет движений)
  - Суббота: Кардио средней интенсивности 30–40 мин
  - Воскресенье: Отдых

Пример базовой силовой программы (под базовый уровень)
- Продажи и подходы: 3–4 подхода по 6–12 повторений, отдых 60–90 секунд между подходами.
- Упражнения (замени на доступное оборудование):
  1) Жим лёжа или гантели на скамье
  2) Тяга в наклоне или тяга в верхнем блоке
  3) Пресс/планка (как основной стабилизатор)
  4) Приседания со штангой или с гантелями
  5) Жим стоя или жим гантелей над головой
  6) Тяга blok/гребля для спины
  7) Подъемы на носки для голени/квадрицепсы

Пример недельного плана (конкретные упражнения можно адаптировать)
- Понедельник (верх): 
  - Жим штанги на скамье 3×8–10
  - Тяга в наклоне 3×8–10
  - Жим гантелей над головой 3×8–12
  - Подтягивания или тяга блока 3×6–10
  - Планка 3×60 сек
- Вторник (кардио): умеренная интервальная 20–30 мин (1 мин работа, 1 мин отдых)
- Среда (низ): 
  - Приседания со штангой 3×8–10
  - Тяга на прямых ногах 3×10–12
  - Выпады 3×10 на каждую ногу
  - Подъемы на носки 3×12–15
  - Лягкие скручивания/боковые планки
- Пятница (полнопрофиль): 
  - Становая тяга или тяжёлая тяга гантелей 3×6–8
  - Жим штанги узким хватом 3×8–10
  - Тяга блока снизу 3×8–12
  - Пресс/мостик 3×12–15
- Суббота (кардио): 30–40 мин умеренно-тяжёлого кардио (бег, велосипед, эллипсоид)

Как следить за прогрессом
- Замеры: вес в одном и том же времени суток, раз в неделю; снимки раз в 2–4 недели.
- Силовые показатели: следи за тем, чтобы вес или количество повторений в упражнениях росли каждые 1–2 недели (или сохранялись без падения с хорошей техникой).
- Восстановление: сон 7–9 часов, минимум 1–2 дня отдыха в неделю.

Питание за 15 секунд (правило)

- Белок на каждой трапезу.
- Овощи и клетчатка каждый день.
- Углеводы вокруг тренировок (до/после занятия — для энергии и восстановления).
- Жиры умеренно, без крайностей.
- Гидратация: 2–3 л воды в день, больше в жару или при активной потере пота.

Важные моменты
- Не нужно резко снижать калории. Если энергия падает или тренировки страдают — скорректируй дефицит вниз или выше.
- Восстановление критично: сон и дни отдыха напрямую влияют на потерю жира и сохранение мышц.
- Если хочешь, могу адаптировать план под твои условия: доступ к залу, оборудование, формат тренировок (домашний зал, двор, абонемент), ограничения по здоровью.

Хочешь, могу рассчитать примерную калорийность и план на первую неделю, исходя из твоих предпочтений и доступного оборудования? Сообщи:
- сколько дней в неделю готов тренироваться
- какая аппаратура есть (зал, гантели, штанга, беговая дорожка и т.д.)
- есть ли ограничения по здоровью
- целевой период для достижения результата (например, 8–12 недель) и желаемый темп снижения веса.

## Создаём класс для ассистента

Чтобы наш ассистент мог поддерживать диалог, необходимо каждый раз запоминать идентификатор предыдущего сообщения. Удобнее создать класс, который будет хранить такой идентификатор внутри себя. Также этот класс будет отвечат за хранение системного промпта и самой модели.

Если мы хотим, чтобы ассистент мог общаться с несколькими пользователями сразу, то надо для каждого пользователя хранить свой идентификатор последнего сообщения. Поэтому будем использовать некоторый идентификатор сессии `session_id`для идентификации пользователя. 

In [15]:
class Assistant:
    def __init__(self, instructions, model=model):
        self.model = model
        self.instructions = instructions
        self.previous_response_id_map = {}

    def __call__(self, input, session_id='default'):
        previous_response_id = self.previous_response_id_map.get(session_id, None)
        res = client.responses.create(
            model = self.model,
            store = True,
            previous_response_id = previous_response_id,
            instructions = self.instructions,
            input = input
        )
        self.previous_response_id_map[session_id] = res.id
        return res.output_text


In [ ]:
instructions = """
Ты - профессиональный фитнес-ассистент. Отвечай как энергичный молодой человек со спортивным задором.
Говори как человек, короткими фразами, избегая перечислений и списков.
"""

assistant = Assistant(instructions)
printx(assistant("Привет! С чего ты порекомендуешь начать тренировки в зале?"))

'Погнали. Начать можно так. Сформируй цель и уровень подготовки. Найди толковую технику у тренера или по проверенным видео. Разминка 5–10 минут: кардио и динамика. Базовый набор на старте: присед, жим лёжа, становая тяга или тяга в наклоне, подтягивания или тяга верхнего блока, жим над головой, работа на пресс. Вес подбирай так, чтобы последние повторения были тяжёлыми, но техника не страдала. Три тренировки в неделю, полное тело за каждый сеанс. По 2–3 рабочих подхода на упражнение, 6–8 повторений для силы или 8–12 для масс. Прогрессия: добавляй вес или повторения каждую неделю. Не забывай про отдых и сон. Хочешь, могу расписать конкретную программу под твой уровень и цель.'

In [ ]:
printx(assistant("Мне не хватает мотивации :("))

Понимаю. Мотивация бывает качелями. Давай сделаем шаги попроще.

На этой неделе цель — просто прийти в зал три раза. Не важен объём, главное начать. Пришёл — сделай 5 минут разминки и пару базовых подходов. Это already движение.

Найди напарника или тренера — ответственность держит. Веди мини-дневник: записывай, что сделал и как почувствовал. Вижу прогресс — вернётся мотивация.

Договорись с собой на маленькое удовольствие после тренировки. Музыка, кофе, сериал — что тебе заходит. Хочешь, придумаю конкретный план под твой график и цель?

: 

## Добавляем Function Calling

Наш агент может вести переписку и давать советы по фитнесу. Добавим к нему полезную функцию - ведения дневника упражнений. Для этого агенту нужно дать возможность писать некоторую информацию в специальную базу данных. Для простоты будем использовать просто список объектов в памяти, хотя на практике, конечно, имеет смысл использвать какую-нибудь СУБД.

Информация о каждом упражнении будем хранить в виде такого объекта:

In [25]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

class Exercise(BaseModel):
    """Эта функция позволяет добавлять информацию о сделанном в зале упражнении."""

    тип: str = Field(description="Тип упражнения (кардио или силовое)", default=None)
    название: str = Field(description="Название упражения", default=None)
    болевые_ощущения : str = Field(description="Болевые ощущения при выполнении упражнения", default=None)
    пульс : int = Field(description="Пульс в момент выполнения упражнения", default=None)
    подходы : int = Field(description="Количество подходов", default=None)
    повторения : int = Field(description="Количество повторений", default=None)

При вызове LLM мы можем указать список возможных функций, которые она может вызвать. Для этого при вызове `responses.create` мы передаём список инструментов `tools`. Это приведёт к тому, что каждый раз при вызове модели ей будет передаваться подсказка, что ей доступна функция с указанным описанием. 

> **ВАЖНО**: Описание семантики функции берётся из doc-строки в классе `Exercise`, а описания параметров функции - из соответствующих полей `description` каждого поля. Поэтому в классе `Exercise` очень важно подробно документировать все поля.

Также, для надёжности, мы опишем критерии вызова функции в системном промпте.

In [28]:
tools = [
    {
        "type": "function",
        "name": "Exercise",
        "description": "Вызывай, когда пользователь сообщает о выполнении упражнения. Сохрани информацию.",
        "parameters": Exercise.model_json_schema(),
    }
]

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Отвечай 
как энергичный молодой человек со спортивным задором. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`.
"""

res = client.responses.create(
    model = model,
    store = True,
    tools = tools,
    instructions = instruction,
    input = "Я сделал 10 приседаний, запиши в дневник"
)

res.to_dict()

{'id': 'resp_68be8df4f464819795c539e37e9e6e8f02c54a56d360a46c',
 'created_at': 1757318644.0,
 'error': None,
 'incomplete_details': None,
 'instructions': '\nТы - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Отвечай \nкак энергичный молодой человек со спортивным задором. Ты можешь\nсоветовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести \nдневник выполненных пользователем упражнений - для этого используй функцию `Exercise`.\n',
 'metadata': {},
 'model': 'gpt-5-nano-2025-08-07',
 'object': 'response',
 'output': [{'id': 'rs_68be8df5c9688197a57016ed3cb297cf02c54a56d360a46c',
   'summary': [],
   'type': 'reasoning'},
  {'arguments': '{"тип":"силовое","название":"приседания","болевые_ощущения":"без боли","пульс":0,"подходы":1,"повторения":10}',
   'call_id': 'call_l1j8p1w1QL7ibyusBXQ11abg',
   'name': 'Exercise',
   'type': 'function_call',
   'id': 'fc_68be8dffa048819788b2ee27f1dd02d602c54a56d360a46c',
   'status': 'completed'}],
 

Мы видим, что в качестве результата вернулся ответ типа `function_call`, и в поле `arguments` у него заполнены параметры функции, извлечённые из запроса пользователя. Теперь нам остаётся реализовать функцию для записи этих данных в некоторую базу данных:

In [29]:
exercise_db = []

def add_exercise(exercise):
    exercise_db.append(exercise)
    return "Упражнение добавлено"

После выполнения функции нам необходимо снова вызвать LLM и передать её результат выполнения функции:

In [31]:
tool_calls = [item for item in res.output if item.type == "function_call"]

if tool_calls:
    out = []
    for call in tool_calls:
        print(f"  • Обрабатываем: {call.name} (call_id={call.call_id})")
        try:
            args = json.loads(call.arguments)
            args = Exercise.model_validate(args)
            result = add_exercise(args)
        except Exception as e:
            result = f"Ошибка: {e}"
        print(f"  • Результат: {result}")
        out.append({
            "type": "function_call_output",
            "call_id": call.call_id,
            "output": result
        })
        res = client.responses.create(
            model=model,
            input=out,
            tools=tools,
            previous_response_id=res.id,
            store=True
        )

printx(res.output_text)

Готово! Записал в дневник:
- Упражнение: приседания
- Тип: силовое
- Подходы: 1
- Повторения: 10
- Болевые ощущения: без боли
- Пульс: не измерял

Хочешь добавить ещё упражнения или уточнить какой-то параметр?

Таким образом, для вызова функции необходимо:
* Сообщить модели о доступных функциях
* При вызове модели обработать функциональный вызов, если в результате вызова модель вернула соответствующий результат 

Для реализации полноценного ассистента нам потребуется ещё добавить функцию распечатки дневника занятий. Поэтому немного структурируем наш код:
* Добавим функцию для обработки вызова прямо в класс с описанием данных
* Создадим класс `Assistant`, который будет реализовывать функциональный вызов, а также автоматически поддерживать диалог, запоминая идентификаторы предыдущих ответов.

> Чтобы ассистент мог поддерживать диалог сразу с несколькими пользователями, нам нужно также ввести некоторый идентификатор сессии `session_id`, и для каждой сессии помнить свою историю переписки и `id` последнего сообщения. По умолчанию будем использовать сессию `default`. 

In [26]:
exercise_db = {}
class Exercise(BaseModel):
    """Эта функция позволяет добавлять информацию о сделанном в зале упражнении."""

    тип: Optional[str] = Field(description="Тип упражнения (кардио или силовое)", default=None)
    название: Optional[str] = Field(description="Название упражения", default=None)
    болевые_ощущения : Optional[str] = Field(description="Болевые ощущения при выполнении упражнения", default=None)
    пульс : Optional[str] = Field(description="Пульс в момент выполнения упражнения", default=None)
    подходы : Optional[str] = Field(description="Количество подходов", default=None)
    повторения : Optional[str] = Field(description="Количество повторений", default=None)

    def process(self, session_id):
        if session_id not in exercise_db:
            exercise_db[session_id] = []
        exercise_db[session_id].append(self)
        return "Упражнение добавлено"

class ListExercises(BaseModel):
    """Эта функция позволяет получить список сделанных упражнений"""

    def process(self,session_id):
        if session_id not in exercise_db:
            return "Упражнений нет"
        else:
            return '\n'.join([
                f"{i+1}. {x.название} ({x.тип}, {x.подходы} подходов, {x.повторения} повторений)" for i,x in enumerate(exercise_db[session_id])
            ])


In [ ]:
class Assistant():
    user_sessions = {}

    def __init__(self, instruction, tools = [], session_id='default', model=model):
        self.instruction = instruction
        self.model = model
        self.tool_map = { x.__name__ : x for x in tools if issubclass(x, BaseModel) }
        self.tools = [
            self._create_tool_annot(x) for x in tools
        ]
        if session_id not in self.user_sessions:
            self.user_sessions[session_id] = {
                "last_reply_id" : None,
                "history" : [],
            }

    def _create_tool_annot(self, x):
        if issubclass(x, BaseModel):
            return {
                "type": "function",
                "name": x.__name__,
                "description": x.__doc__,
                "parameters": x.model_json_schema(),
            }
        else:
            return x


    def __call__(self, message, session_id='default'):
        s = self.user_sessions[session_id]
        s['history'].append({ 'role': 'user', 'content': message })
        res = client.responses.create(
            model = self.model,
            store = True,
            tools = self.tools,
            instructions = self.instruction,
            input = message
        )
        tool_calls = [item for item in res.output if item.type == "function_call"]
        if tool_calls:
            s['history'].append({ 'role' : 'func_call', 'content' : res.output_text })
            out = []
            for call in tool_calls:
                print(f"  • Обрабатываем: {call.name} (call_id={call.call_id})")
                try:
                    fn = self.tool_map[call.name]
                    obj = fn.model_validate(json.loads(call.arguments))
                    result = obj.process(session_id)
                except Exception as e:
                    result = f"Ошибка: {e}"
                print(f"  • Результат: {result}")
                out.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
                res = client.responses.create(
                    model=self.model,
                    input=out,
                    tools=self.tools,
                    previous_response_id=res.id,
                    store=True
                )
        self.user_sessions[session_id]['last_reply_id'] = res.id
        s['history'].append({ 'role' : 'assistant', 'content' : res.output_text })
        return res


In [ ]:
instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant(instruction, [Exercise, ListExercises])

printx(assistant('Привет! Посоветуй, как начать тренироваться в зале!').output_text)

Отлично! Начать легко и грамотно можно с простого базового плана. Ниже — практичный старт на первые 4–6 недель, а затем идеи для прогрессии. Если хочешь, могу адаптировать под твой график и оборудование.

Ключевые принципы для новичка
- Техника важнее веса. Ставь форму превыше всего.
- Прогрессия постепенно: добавляй вес или повторения каждые 1–2 недели.
- 3 тренировки в неделю достаточно на старте. Каждая сессия — полноценная тренировка всего тела.
- Разминка 8–10 минут: лёгкая кардиоразминка + динамические движения для плеч, тазобедренных и спины.
- Восстановление: 7–9 часов сна, достаточное питание, водный баланс.
- Пропорции в питании: ориентировочно 1.6–2.2 г белка на кг массы тела в день; умеренный дефицит/избыток калорий в зависимости от цели.

Пример базовой программы (3 дня в неделю,全体)
Структура: 3 подхода по 8–12 повторений. Между подходами 60–90 секунд.

День 1
- Присед со штангой (или гоблет-присед): 3x8–12
- Жим лёжа на скамье (или отжимания от пола): 3x8–12
- Тяга в наклоне или тяга верхнего блока: 3x8–12
- Планка: 3x20–40 секунд

День 2
- Выпады/шаги вперёд с гантелями: 3x8–12 на каждую ногу
- Жим гантелей над головой: 3x8–12
- Тяга гири/гантели к поясу в наклоне: 3x8–12
- Гиперэкстензия или подъем корпуса на пресс: 3x12–15

День 3
- Становая тяга (романтианская или классическая, если техника хороша) или «мертвая тяга» с гирями: 3x8–12
- Жим гантелей на наклонной скамье: 3x8–12
- Подтягивания или тяга вертикального блока: 3x8–12
- Русские скручивания или велосипед-кроник: 3x12–20 на каждую сторону

Как прогрессировать
- Если можешь выполнить 12 повторений уверенно на протяжении двух тренировок подряд, добавь вес на следующей сессии (обычно 2,5–5 кг на большие упражнения).
- Либо добавляй по 1–2 повторения в каждом подходе до верхней границы, затем увеличивай вес.
- Каждую 4–6 недель можно сделать легкую смену упражнений (например, заменить жим на жим incline, заменить присед на leg press) чтобы мышцы «абонентски» адаптировались.

Дополнительные советы
- Разминка и техника: держи спину нейтральной, грудная клетка вверх, взгляд чуть ниже; дыши так, чтобы не задерживать дыхание во время усилия.
- Варианты под рукой: если у тебя нет доступа к штанге — заменяй на машинные версии или гантели: присед с гантелями, жим гантелей на скамье, тяга в тренажере, выпады с гантелями.
- Кардио: 1–2 лёгкие кардио-сессии по 20–30 минут в неделю для сердечно-сосудистого здоровья.
- Безопасность: если есть боли (неожиданные или резкие), остановись и проконсультируйся со специалистом.

Питание и восстановление (быстрые ориентиры)
- Белок: примерно 1.6–2.2 г на кг массы тела в день.
- Калории: подстрой под цель — поддержка, набор массы или похудение.
- Гидратация: 2–3 литра воды в день (больше при тренировках).
- Сон: 7–9 часов ночью.

Хочешь, чтобы я адаптировал план под твой график и доступное оборудование (есть ли свободные веса, тренажеры, кардио-зона)? Могу также составить конкретный 4–6 недельный план с точными упражнениями и весами в начале каждого цикла.

Готов начать прямо сейчас — скажи, сколько дней в неделю можешь тренироваться и что есть из оборудования в зале. Также могу начать вести дневник твоих занятий: просто скажи, какие упражнения сделал(а), вес и повторения, и я запишу их как твой журнал тренировок.

In [ ]:
printx(assistant('Я сделал 10 приседаний, запиши!').output_text)

  • Обрабатываем: Exercise (call_id=call_kDJDskM7zBthN0CjF9Rsozhi)
  • Результат: Упражнение добавлено


Готово. Запись добавлена:
- Тип: силовое
- Название: Приседания
- Подходы: 1
- Повторения: 10
- Болевые ощущения: нет
- Пульс: не указан

Хочешь добавить пульс, время выполнения или записать еще упражнения/серии?

In [ ]:
printx(assistant('Напомни, какие я сделал упражнения?').output_text)

  • Обрабатываем: ListExercises (call_id=call_I9JpNpvXuTb6zlIWMv28yO0A)
  • Результат: 1. Приседания (силовое, 1 подходов, 10 повторений)


Вот что у меня записано:

- Приседания — силовое, 1 подход, 10 повторений.

Хочешь добавить детали (пульс, боли, дату) или занести новое упражнение? Могу моментально это сделать.

## Добавляем RAG

Наш ассистент сам по себе неплохо отвечает на вопросы по тренировкам, однако если мы хотим сделать на основе бота консультанта конкретного фитнес-клуба, который должен обладать конкретными знаниями о его работе - необходимо дополнить его знания специализированной информацией. Для этого мы можем использовать RAG - подход, который позволяет искать релевантную информацию по запросу в текстовых базах знаний.

В качестве текстовой базы знаний в нашем примере возьмём несколько популярных текстов о фитнесе:

In [6]:
from glob import glob
import pandas as pd
import tiktoken

def get_token_count(filename):
    tokenizer = tiktoken.encoding_for_model(model)
    with open(filename, "r", encoding="utf8") as f:
        return len(tokenizer.encode(f.read()))

def get_file_len(filename):
    with open(filename, encoding="utf-8") as f:
        l = len(f.read())
    return l

d = [
    {
        "File": fn,
        "Tokens": get_token_count(fn),
        "Chars": get_file_len(fn),
    }
    for fn in glob("data/text-kb/*.txt")
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars
0,data/text-kb\diary.txt,247,931
1,data/text-kb\how-to-begin.txt,811,3249
2,data/text-kb\program.txt,231,937
3,data/text-kb\qna.txt,1712,7098
4,data/text-kb\what-to-take.txt,262,954


В случае с RAG текстовая база знаний делится на небольшие фрагменты, по которым в ходе запроса осуществляется поиск. Оптимальная длина фрагмента определяется опытным путём, но в среднем разумно придерживаться размера 1000-2000 токенов на фрагмент. В нашем случае длина некоторых файлов превышает 1000 токенов, поэтому мы будем использовать **стратегию чанкования**, для разбиения текстовых файлов на более мелкие фрагементы.

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файлов, нам необходимо построить индекс, а перед этим - загрузить все файлы в облако.

In [9]:
def upload_file(filename):
    return client.files.create(file=open(filename,'rb'),purpose='assistants')

df["Uploaded"] = df["File"].apply(upload_file)

## Строим векторный индекс

Для индексации файлов необходимо создать векторный индекс (vector store), и поместить все файлы туда:


In [20]:
vector_store = client.vector_stores.create(name='rag_store')

def add_to_store(file):
    client.vector_stores.files.create(
        vector_store_id=vector_store.id, 
        file_id=file.id,
        chunking_strategy={
            "type": "static",
            "static" : { "max_chunk_size_tokens" : 1000, "chunk_overlap_tokens" : 100 }
        }
        )

_ = df['Uploaded'].apply(add_to_store)

Теперь мы можем искать внутри нашего векторного хранилища:

In [21]:
res = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="С чего начать занятия в зале?"
)
for x in res.data:
    print(f"{len(x.content[0].text)} символов из файла {x.filename}, релевантность = {x.score}")

2128 символов из файла how-to-begin.txt, релевантность = 0.8512098460553149
2154 символов из файла qna.txt, релевантность = 0.8274196502149863
1345 символов из файла how-to-begin.txt, релевантность = 0.7430554808995832
947 символов из файла program.txt, релевантность = 0.7269855696420007
941 символов из файла diary.txt, релевантность = 0.7166376431964977
2116 символов из файла qna.txt, релевантность = 0.6892210470689067
2240 символов из файла qna.txt, релевантность = 0.6883783965528538
962 символов из файла what-to-take.txt, релевантность = 0.6662310286448695
1199 символов из файла qna.txt, релевантность = 0.18211388622259136


## Собираем RAG-ассистента

Для того, чтобы дать нашему ассистенту возможность искать в векторной базе данных, нам необходимо передать дополнительно **инструмент поиска**. Также важно задать хорошую инструкцию (системный промпт): 

In [43]:
search_tool = {
    "type" : "file_search",
    "vector_store_ids" : [vector_store.id]
}

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Отвечай на основе имеющейся
дополнительной информации из файловой базы знаний. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant(instruction, [search_tool, Exercise, ListExercises])

res = assistant("Что нужно, чтобы начать заниматься в зале?")
printx(res.output_text)

Отличный вопрос. Вот базовый чек-лист, что нужно, чтобы начать тренироваться в зале и делать первые шаги безопасно и эффективно.

- Определите цель и оцените здоровье
  - Подумайте, зачем вы идёте в зал (похудение, набор массы, выносливость, общая физическая форма) и оцените состояние здоровья. Это поможет выбрать направление тренировки и нагрузок. Важно понять, что на начальном этапе цель и состояние здоровья определят программу и частоту занятий. Рекомендации начинающих начинаются с постановки целей и оценки здоровья. 

- Пройдите вводный инструктаж/консультацию с тренером
  - В первый визит запишитесь на вводный инструктаж: тренер объяснит работу тренажёров, технику и зону работы в зале. Это почти всегда входит в стоимость абонемента. 
  - Для безопасности и эффективности лучше начинать с консультаций и последующей помощи тренера, особенно если вы новичок. 

- Определите базовый план тренировок
  - Придумайте недельный план на каждую группу мышц, с учётом вашего расписания. Начинаем с реалистичной частоты и объёмов и по мере прогресса flexibel адаптируем. 
  - Для новичков часто полезны круговые или умеренно-силовые подходы с акцентом на технику и восстановление. 

- Разминка и техника прежде веса
  - Хорошая разминка на 10 минут, чтобы подготовить мышцы и суставы. 
  - Начинайте с небольших весов и высокой техникой; цель — 2–3 подхода по 8–12 повторений, отдых 2–3 минуты между подходами. Со временем веса можно увеличивать. 
  - Техника важнее веса: сначала вы учитесь выполнять упражнение правильно, затем наращиваете нагрузку. Не стремитесь “бить максимум” с первых занятий. 

- Частота занятий и прогресс
  - Для новичков часто рекомендуют 2–3 тренировки в неделю, с постепенным увеличением нагрузки. 
  - Не перегружайте себя на первых тренировках: важна регулярность и постепенность. 

- Что взять с собой на тренировку
  - Удобная спортивная одежда и обувь, бутылка воды, полотенце, средства гигиены и т. д. Подготовьте всё по списку, чтобы ничто не отвлекало. 

- Поддержка и мотивация
  - Ведите дневник тренировок: фиксируйте упражнения, вес, повторения, подходы, время отдыха и общее время тренировки. Это помогает отслеживать прогресс и оставаться мотивированным. 

- Возможные противопоказания и осторожности
  - Есть противопоказания и предупреждения для занятий спортом; при хронических или серьёзных заболеваниях лучше обсудить план с врачом и тренером. 

Хочете, я помогу вам быстро собрать стартовый план под ваши цели и расписание? Могу предложить простой 3-дневный план на неделю (например: день 1 — спина/грудь, день 2 — ноги/ягодицы, день 3 — плечи/руки) с базовыми упражнениями и прогрессией. Также могу завести для вас дневник тренировок и начать фиксировать первые занятия.

Посмотрим, из каких источников был получен этот ответ:

In [45]:
[ x for x in res.output ]

[ResponseReasoningItem(id='rs_68c06782d4d0819389485e499bd17fca0f345fbcd22359cd', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseFileSearchToolCall(id='fs_68c067860a3881939aad2b860870a3ce0f345fbcd22359cd', queries=['Что нужно, чтобы начать заниматься в зале?', 'начать занятия в тренажерном зале рекомендации', 'что необходимо для новичка в зале', 'первый визит в зал базовый комплект', 'начальный план тренировок новичка зал'], status='completed', type='file_search_call', results=None),
 ResponseReasoningItem(id='rs_68c06787e5f08193a89dea9db5a3b4c90f345fbcd22359cd', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseOutputMessage(id='msg_68c067913dbc819383d4b98a496e6a8c0f345fbcd22359cd', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-QiUtwm8ULLRftrtqxzJqtB', filename='qna.txt', index=526, type='file_citation'), AnnotationFileCitation(file_id='file-QiUtwm8ULLRftrtqxzJqtB', file

In [54]:
from openai.types.responses.response_output_message import ResponseOutputMessage

def find_obj(t,l):
    for x in l:
        if isinstance(x,t):
            return x
    return None

def print_citations(result):
    o = find_obj(ResponseOutputMessage,result.output)
    for x in o.content[0].annotations:
        if x.type == "file_citation":
            print(f"{x.filename}, idx={x.index}")

print_citations(res)

qna.txt, idx=526
qna.txt, idx=748
qna.txt, idx=872
program.txt, idx=1075
qna.txt, idx=1185
how-to-begin.txt, idx=1289
how-to-begin.txt, idx=1451
how-to-begin.txt, idx=1604
qna.txt, idx=1731
qna.txt, idx=1816
what-to-take.txt, idx=1994
diary.txt, idx=2202
qna.txt, idx=2396


## Добавляем таблицу добавок

Для серьезных тренировок важны также пищевые добавки. Чтобы добавить информацию о них в нашего ассистента, мы нашли таблицу таких добавок в формате markdown:

In [57]:
with open("data/additives.md", encoding="utf-8") as f:
    additives = f.readlines()
additives_all = "".join(additives)

tokenizer = tiktoken.encoding_for_model(model)
tokens = len(tokenizer.encode(additives_all))
print(f"Токенов: {tokens}, {len(additives_all)/tokens} chars/token")

Токенов: 6833, 4.82218644811942 chars/token


In [58]:
printx(additives_all[:1040])

| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |
|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|
| Креатин                                          | Для наращивания мышечной массы, силы и ускорения восстановления | Увеличивает физическую силу, ускоряет восстановление; увеличивает массу                                   | 2-20 г в день                 | *****   |
| Глютамин                                         | Для наращивания мышечной массы, силы и ускорения восстановления | Предотвращает распад мышечной ткани; укрепляет иммунную систему                                           | 5-20 г в день                 | *****   

Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [60]:
header = additives[:2]
header

['| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |\n',
 '|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [70]:
import io

chunk_size = 600 * 5  # около 600 tokens * 5 char/token

s = header.copy()
uploaded_additives = []
i = 0
for x in additives[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        f = client.files.create(
            purpose="assistants",
            file = (f'table_{i}.md',io.BytesIO("".join(s).encode("utf-8")),'text/markdown')
        )
        client.vector_stores.files.create(file_id=f.id, vector_store_id=vector_store.id)
        uploaded_additives.append(f)
        i+=1
        s = header.copy()
print(f"Uploaded {len(uploaded_additives)} table chunks")

Uploaded 12 table chunks


Посмотрим, как система стала отвечать на вопросы о добавках:

In [71]:
res = assistant("Какие добавки помогают нарастить мышечную массу?")

printx(res.output_text)
print_citations(res)

Коротко: для прироста мышечной массы в первую очередь важны качественное питание и нормальная тренировка. Добавки могут помочь ускорить прогресс, но без адаптации рациона и силовых нагрузок они работают слабее. Ниже — наиболее часто упоминаемые в ваших файлах варианты с примечаниями по дозировке и доказательством в вашей базе знаний.

- Креатин моногидрат
  - Что делает: повышает силу и ускоряет восстановление, что способствует росту мышечной массы.
  - Как принимать: диапазон дозировки в ваших файлах — примерно 2–20 г в день (чаще используют около 3–5 г в день в постоянном режиме; иногда бывает кратковременная загрузка). Источник: таблица с перечислением креатина и диапазона дозировки. fileciteturn0file0turn0file2

- Белковые концентраты (молочный/сывороточный/казеин и т. д.)
  - Что делают: обеспечивают аминокислотный пакет для роста мышц и восстановления.
  - Как принимать: в таблицах указано 20–200 г белка в день в зависимости от рациона и цели. Это часть ежедневной протеиновой поддержки. Источник: таблица с белковыми концентратами. 
  - Примечание: если не хватает белка в диете, используйте протеин после тренировки. Об этом прямо упомянуто в обзоре вопросов и ответов. 

- BCAA (аминокислоты с разветвлёнными цепями)
  - Что делают: помогают предотвратить распад мышц и снижают усталость во время тренинга.
  - Как принимать: 5–10 г в день. Источник: таблица по BCAA в вашей базе. 

- Глютамин
  - Что делает: поддерживает мышечный анаболизм и восстановление.
  - Как принимать: 5–20 г в день. Источник: та же таблица по глютамину. 

- NMV (гидроксиметилбутират, HMB)
  - Что делает: потенциально поддерживает рост мышечной массы и уменьшает распад белка.
  - Как принимать: 3–5 г в день. Источник: таблица по NMV. 

- ZMA (цинк-монометионин-аспартат + магний + B6)
  - Что делает: заявлено увеличение тестостерона/IGF-1, улучшение сна и восстановления.
  - Как принимать: 2–3 капсулы в день. Источник: таблица по ZMA. 

- Аргинин и его вариации (АКИЦ и другие)
  - Аргинин: 5–15 г в день — может стимулировать рост гормона и поддерживать мышечный анаболизм.
  - АКИЦ: 4–6 г в день — предотвращает распад белка; мозговая активность также может улучшаться.
  - Источники: таблицы по Аргинину и AKiС. 

- α-кетоглутаратные соединения и производные (AKG/OKG)
  - OKG: 6–8 г в день — поддерживает анаболизм и предотвращает распад ткани.
  - AKG: 3–4 г в день — аналогично поддерживает мышечный белок.
  - Источники: таблицы по AKG и OKG. 

- Прочие добавки (для контекста)
  - В ряде таблиц встречаются также ornithine, OKG, орнитин и другие соединения, которые претендуют на поддержку анаболизма и восстановления, с различной научной обоснованностью. Если хотите, могу подобрать наиболее разумные из них под ваш уровень подготовки и цели — скажите, какие именно варианты рассматриваете.
  - В качестве предупреждения: жиросжигатели и некоторые экстремальные добавки могут повышать нагрузку на сердце и нарушать сон; используйте их осторожно и только после консультации с врачом. Это отражено в разделах вопросов и ответов в вашем файле. 

Как правильно подойти к выбору и применению добавок
- Ваша база указывает на то, что базовыми вещами остаются достаточное потребление белка и калорий, а также качественная силовая тренировка. Добавки работают на фоне этих факторов и не заменяют их. Если цель — масса, обоснованно пользуйтесь креатином и протеином как базовыми инструментами, а остальные добавки — на выбор по сочетанию вкусов, переносимости и индивидуальных целей. 
- Готовы подобрать конкретный набор под ваш уровень подготовки, вес, дневной рацион и цели? Расскажите:
  - ваш текущий вес, рост и цель (набор массы vs. рельеф);
  - обычный дневной рацион и примерный уровень белка;
  - есть ли проблемы со здоровьем или ограничения (сердцу, почкам и т. п.).

Если хотите, могу прямо сейчас составить для вас минимально необходимый набор добавок с примерной схемой приема на неделю и учесть ваши предпочтения. Также могу вести дневник упражнений и прогресса — скажите, хотите ли начать запись и какие тренировки сейчас делаете.

table_6.md, idx=1057
qna.txt, idx=1196
table_0.md, idx=1408
table_0.md, idx=1559
table_0.md, idx=1742
table_0.md, idx=1946
table_2.md, idx=2226
table_0.md, idx=2461
qna.txt, idx=3061
qna.txt, idx=3495


## Удаляем лишнее

В заключение удалим созданные ресурсы!

> **ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие файлы или поисковые индексы в проекте!  

In [72]:
vector_stores = client.vector_stores.list()
for v in vector_stores:
    print(f" + Deleting vector store id={v.id}")
    client.vector_stores.delete(vector_store_id=v.id)

files = client.files.list(purpose='assistants')
for f in files:
    print(f" + Deleting file id={f.id}")
    client.files.delete(file_id=f.id)

 + Deleting vector store id=vs_68c05ae6dfb48191ba2d825dfb4b40dc
 + Deleting vector store id=vs_68c05a9d567c8191930aa77680d6ccd6
 + Deleting vector store id=vs_68c05a173d048191b72312f9d50ef54c
 + Deleting vector store id=vs_68c058ec59a08191ba058f6e0e1d8bd6
 + Deleting file id=file-HRiSxGaE7VfNMk2CzfZtZo
 + Deleting file id=file-MjjbkkC5HD9HRSmQSQQpT7
 + Deleting file id=file-ESK3JQ5gWQMMby62U6piyG
 + Deleting file id=file-GtR8YNeAA1StqhzKrnkGRc
 + Deleting file id=file-TXHnQP2NT8x9XP4n4tuFjw
 + Deleting file id=file-LbhNajaWMZaPQEF28zR8cf
 + Deleting file id=file-Y4ChZMU95S9V8J6vumzgr3
 + Deleting file id=file-WeyrdfmPUyKT2puZLz4vc3
 + Deleting file id=file-L1mLyVH3qXzaCnihAoB7gh
 + Deleting file id=file-YQSFygEJCk2p2AvC9Yjcgr
 + Deleting file id=file-4s7d9LZg2fCH1djtmt1pR4
 + Deleting file id=file-6R1Kw713mi7N4v9WQfv7SC
 + Deleting file id=file-AKvcrc2ke4HcEN913TRLwk
 + Deleting file id=file-NuFMW2wowg1XK9cvytKeZr
 + Deleting file id=file-QMpTxUm79v95dgARH8zhYh
 + Deleting file id=file